### Day 4: Switching to a Class For Reusability

Still ugly, but makes modifying certain conditions a bit clearer (for me!).

#### Part 1: 

Need to have 7 of the 8 keys 

#### Part 2

Now just need a function to check each key in the dict aligning with rules provided: 

```
byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
```

### Architecture: 

- build a passport validator class
- take in data & run the various checks:
    - 1) check keys only (option passed in)
    - 2) check values:
        - each type of condition has a separate method

In [1]:
import re

class passportValidator():
    def __init__(self, rules, passDict, keys_only):
        self.rules = rules
        self.passDict = passDict
        self.keys_only = keys_only
        self.valid = False
        self.year_len = 4 # length of year chars
        self.ecl_list = ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
    
    def __runKeyCheck(self):
        """Need all 8 fields, allowed to not have so 7 total fields needed"""

        if len(set(self.passDict.keys()) & set(self.rules)) < 7:
            return False
        else:
            return True
        
    def __runValueCheck(self):
        """Run through all required fields & confirm proper keys"""
        
        # start by checking years - if false, then return false which ends script
        if not self.__checkYears():
            return False
        
        # check hgt
        if not self.__checkHgt():
            return False

        # check hcl
        if not self.__checkHcl():
            return False

        # check ecl
        if not self.__checkEcl():
            return False     

        # check pid 
        if not self.__checkPid():
            return False

        # if we made it this far, then it is clean!
        return True
        
    def __checkYears(self):
        """Check all years which includes byr, iyr, eyr"""

        # byr check
        if len(self.passDict['byr']) != self.year_len or not (1920 <= int(self.passDict['byr']) <= 2002):
            return False

        # iyr check
        if len(self.passDict['iyr']) != self.year_len or not (2010 <= int(self.passDict['iyr']) <= 2020):
            return False

        # eyr check
        if len(self.passDict['eyr']) != self.year_len or not (2020 <= int(self.passDict['eyr']) <= 2030):
            return False
        
        return True
    
    def __checkHgt(self):
        """Check the hgt key. Try is to capture bad inputs"""
        try:
            match = re.match(r"([0-9]+)([a-z]+)", self.passDict['hgt'], re.I)
            hgt, unit = match.groups()
            hgt = int(hgt)
        except:
            return False

        # now follow conditions for height
        # TODO: Could parametrize the various height values - this is ugly still
        if unit == "cm":
            if not (150 <= hgt <= 193):
                return False
            else:
                return True
        elif unit == "in":
            if not (59 <= hgt <= 76):
                return False
            else:
                return True
        else:
            return False
        
    def __checkHcl(self):
        """Check hcl key"""
        symbol = self.passDict['hcl'][0]
        color = self.passDict['hcl'][1:]

        # check symbol and length of color - tested
        if symbol != "#" or len(color) != 6:
            return False

        # using search to see if any chars in string fall outside of range - tests passed 
        excluded = re.compile('[^a-fA-F0-9]').search
        if bool(excluded(color)):
            return False
        
        return True
    
    def __checkEcl(self):
        """ Check Ecl Key"""
        if self.passDict['ecl'] not in self.ecl_list:
            return False
        
        return True
    
    def __checkPid(self):
        """df"""
        if self.passDict['pid'].isnumeric() and len(self.passDict['pid']) == 9:
            return True
        return False
        
    def returnResult(self):
        """Run proper form of validation depending on keys_only or keys & values"""
        if self.keys_only:
            if self.__runKeyCheck():
                self.valid = True
        else:
            if self.__runKeyCheck():
                if self.__runValueCheck():
                    self.valid = True

        return self.valid

In [2]:
# Required Keys: 
rules = ['ecl', 'pid', 'eyr', 'hcl', 'byr', 'iyr', 'hgt']

# Read in data
with open('day04.txt') as fh:
    lines = fh.readlines()

text = [line.strip() for line in lines]

# build a clean list to store all elements needed, eventually convert to dict
clean_list = []
intermediate_list = []
for line in text:

    # catching the breaks
    if line != '':
        intermediate_list.append(line)
    else:
        # append whatever we have
        clean_list.append(intermediate_list)
        
        # wipe away list
        intermediate_list = []
        

# make sure to append the last line
clean_list.append(intermediate_list)

# tracking
validpt1 = 0
validpt2 = 0

# iterate through list of passports
for passport in clean_list:
    
    # use join to connect strings together for single passport
    clean_passport = ' '.join((info for info in passport))
    
    # split string based on spacing first 
    key_val = clean_passport.split()
    
    # thank you python - dictionary comprehension time 
    keyval_dict = {key.split(":")[0]:key.split(":")[1] for key in key_val}
    
    # pass into function
    
    objectDict = passportValidator(rules, keyval_dict, True)
    if objectDict.returnResult():
        validpt1 += 1
    
    # Part 2 run - check keys & values
    objectDict = passportValidator(rules, keyval_dict, False)
    if objectDict.returnResult():
        validpt2 += 1
        
print(f"Total valid passports for Part 1: {validpt1}")
print(f"Total valid passports for Part 2: {validpt2}")

Total valid passports for Part 1: 237
Total valid passports for Part 2: 172
